In [1]:
######################################################################################################
## Etapa 0: Importando as bibliotecas necessárias
######################################################################################################
import os
import time

from concurrent.futures import ThreadPoolExecutor
from src.modules.image import Image, display_image
from src.modules.detection import Detection, display_image_with_detections
from src.modules.canny import Canny, display_canny
from src.modules.adjacency_matrix import AdjacencyMatrix
from src.modules.connected_components import ConnectedComponents, highlight_components
from src.modules.mst import MinimunSpanningTree
from src.modules.build_tree import BuildTree, plot_logest_path
from src.modules.sort_points import SortPoints, plot_sorted_points
from src.modules.compute_splines import ComputeSplines, plot_splines

import random 
random.seed(42)

In [2]:
def process_image(file_path):
    try:
        start_step1 = time.time()
        print("Etapa 1: Leitura da imagem")
        img = Image(file_path)
        img.reading_image()
        end_step1 = time.time()


        # Etapa 2: Detecção de características
        print("Etapa 2: Detecção de características")
        detection = Detection(img.gray_image)
        detection.compute_all_detections()
        end_step2 = time.time()
        # print(detection.nose.x, detection.nose.y)


        # Etapa 3: Detecção de bordas
        print("Etapa 3: Detecção de bordas")
        canny = Canny(detection.mouth.img, detection.mouth.x, detection.mouth.y,
                      detection.nose.img, detection.nose.x, detection.nose.y,
                      detection.left_eye.img, detection.left_eye.x, detection.left_eye.y,
                      detection.right_eye.img, detection.right_eye.x, detection.right_eye.y,
                    #   detection.mid_point,
                    #   detection.mouth.mid_point, detection.nose.mid_point, detection.left_eye.mid_point, detection.right_eye.mid_point
                      )
        canny.compute_all_edges()
        end_step3 = time.time()


        # Etapa 4: Criação das matrizes de adjacência
        print("Etapa 4: Criação das matrizes de adjacência")
        adjacency_matrix = AdjacencyMatrix(canny.mouth_dict, canny.nose_dict, canny.left_eye_dict, canny.right_eye_dict)
        adjacency_matrix.compute_all_matrices()
        end_step4 = time.time()

        # Etapa 5: Cálculo de componentes conectados para cada matriz
        print("Etapa 5: Cálculo de componentes conectados para cada matriz")
        components_results = ConnectedComponents(adjacency_matrix.nose_adjacency_matrix, adjacency_matrix.mouth_adjacency_matrix, adjacency_matrix.left_eye_adjacency_matrix, adjacency_matrix.right_eye_adjacency_matrix)
        components_results.compute_all_components()
        components_results.compute_main_cc()
        end_step5 = time.time()
        # Etapa 6: Cálculo da árvore geradora mínima
        print("Etapa 6: Cálculo da árvore geradora mínima")
        mst = MinimunSpanningTree(components_results.dic_main_nose_cc, components_results.dic_main_mouth_cc, components_results.dic_main_left_eye_cc, components_results.dic_main_right_eye_cc,
                                  adjacency_matrix.nose_adjacency_matrix, adjacency_matrix.mouth_adjacency_matrix, adjacency_matrix.left_eye_adjacency_matrix, adjacency_matrix.right_eye_adjacency_matrix)
        mst.compute_all_mst()
        end_step6 = time.time()

        # Etapa 7: Construção da árvore e poda
        print("Etapa 7: Construção da árvore e poda")
        tree = BuildTree(mst.min_trees_nose, mst.min_trees_mouth, mst.min_trees_left_eye, mst.min_trees_right_eye)
        tree.compute_all_trees()
        end_step7 = time.time()

        # Etapa 7: Construção da árvore e poda
        print("Etapa 8: Ordenando os pontos")
        sort = SortPoints(tree.new_longest_path_nose, tree.new_longest_path_mouth, tree.new_longest_path_left_eye, tree.new_longest_path_right_eye,
                          mst.nodes_nose, mst.nodes_mouth, mst.nodes_left_eye, mst.nodes_right_eye,
                          canny.nose_dict, canny.mouth_dict, canny.left_eye_dict, canny.right_eye_dict)
        sort.compute_all_coordenates()


        # Etapa 8: Splines
        print("Etapa 8: Splines")
        splines = ComputeSplines(sort.sorted_coord_nose, sort.sorted_coord_mouth, sort.sorted_coord_left_eye, sort.sorted_coord_right_eye)
        splines.compute_all_splines()
        end_step8 = time.time()

        

        print(f"Processado: {file_path}")
        print(f"Tempo1 {end_step1 - start_step1}\nTempo2 {end_step2 - end_step1}\nTempo3 {end_step3 - end_step2}\nTempo4 {end_step4 - end_step3}\nTempo5 {end_step5 - end_step4}\nTempo6 {end_step6 - end_step5}\nTempo7 {end_step7 - end_step6}\nTempo8 {end_step8 - end_step7}")
        return img, detection, canny, adjacency_matrix, components_results, mst, tree, sort, splines

    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")
        return None

    
    
directory = './unzip_images'

file_paths = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(('.ppm'))]

with ThreadPoolExecutor(max_workers=5) as executor:
    results = list(executor.map(process_image, file_paths))
    # quero guardar os 3 primeiros strings do file_path
    tag = [os.path.basename(file_path).split('.')[0].split('_')[0] for file_path in file_paths]

    results_tag = []
    for i in range(len(results)):
        if results[i] is not None:
            results_tag.append((f"{tag[i]}_{i}", results[i]))



Etapa 1: Leitura da imagemEtapa 1: Leitura da imagem

Etapa 1: Leitura da imagem
Etapa 1: Leitura da imagem
Etapa 1: Leitura da imagem
Etapa 2: Detecção de características
Etapa 2: Detecção de características
Etapa 2: Detecção de características
Etapa 2: Detecção de características
Etapa 2: Detecção de características
Etapa 3: Detecção de bordas
Etapa 4: Criação das matrizes de adjacência
Etapa 5: Cálculo de componentes conectados para cada matriz
Etapa 6: Cálculo da árvore geradora mínima
Etapa 3: Detecção de bordas
Etapa 3: Detecção de bordas
Etapa 4: Criação das matrizes de adjacência
Etapa 4: Criação das matrizes de adjacência
Etapa 5: Cálculo de componentes conectados para cada matrizEtapa 5: Cálculo de componentes conectados para cada matriz

Etapa 3: Detecção de bordas
Etapa 4: Criação das matrizes de adjacência
Etapa 7: Construção da árvore e poda
Etapa 5: Cálculo de componentes conectados para cada matriz
Etapa 3: Detecção de bordas
Etapa 8: Ordenando os pontos
Etapa 8: Spline

In [3]:
results_tag[0][1][1].nose.mid_point

(135, 157)

In [4]:
import numpy as np

def points(left_eye_x, right_eye_x, nose_x, mouth_x,
           left_eye_y, right_eye_y, nose_y, mouth_y):
    # Adiciona os pontos da boca
    le_x = np.concatenate([arr.flatten() for arr in left_eye_x]).tolist()
    re_x = np.concatenate([arr.flatten() for arr in right_eye_x]).tolist()
    m_x = np.concatenate([arr.flatten() for arr in mouth_x]).tolist()
    n_x = np.concatenate([arr.flatten() for arr in nose_x]).tolist()

    le_y = np.concatenate([arr.flatten() for arr in left_eye_y]).tolist()
    re_y = np.concatenate([arr.flatten() for arr in right_eye_y]).tolist()
    m_y = np.concatenate([arr.flatten() for arr in mouth_y]).tolist()
    n_y = np.concatenate([arr.flatten() for arr in nose_y]).tolist()

    x = np.concatenate([le_x, re_x, m_x, n_x]).tolist()
    y = np.concatenate([le_y, re_y, m_y, n_y]).tolist()

    points1 = list(zip(x, y))

    nose_points = list(zip(n_x, n_y))
    mouth_points = list(zip(m_x, m_y))
    left_eye_points = list(zip(le_x, le_y))
    right_eye_points = list(zip(re_x, re_y))



    return points1, left_eye_points, right_eye_points, nose_points, mouth_points
    

In [5]:
import pandas as pd
df_people_points = pd.DataFrame(columns=['tag', 'points'])

for i in range(len(results_tag)):
    teste = points(results_tag[i][1][8].all_X_left_eye, 
                results_tag[i][1][8].all_X_right_eye,
                results_tag[i][1][8].all_X_nose,
                results_tag[i][1][8].all_X_mouth,
                results_tag[i][1][8].all_Y_left_eye,
                results_tag[i][1][8].all_Y_right_eye,
                results_tag[i][1][8].all_Y_nose,
                results_tag[i][1][8].all_Y_mouth)
    tag = results_tag[i][0]
    df_people_points = pd.concat([df_people_points, pd.DataFrame({'tag': [tag], 'points': [teste], 'left_eye_points': [teste[1]], 'right_eye_points': [teste[2]], 'nose_points': [teste[3]], 'mouth_points': [teste[4]]})], ignore_index=True)


In [6]:
def dtw_2d_curve(series1, series2):
    n, m = len(series1), len(series2)
    dtw_matrix = np.full((n + 1, m + 1), np.inf)
    dtw_matrix[0, 0] = 0

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = np.linalg.norm(np.array(series1[i-1]) - np.array(series2[j-1]))
            dtw_matrix[i, j] = cost + min(
                dtw_matrix[i - 1, j],    # inserção
                dtw_matrix[i, j - 1],    # remoção
                dtw_matrix[i - 1, j - 1] # match
            )

    return dtw_matrix[n, m]


In [7]:
from collections import Counter

# Função para calcular distâncias e pegar tag mais comum
def get_most_common_tag_for_each(df_people_points):
    n = len(df_people_points)
    results = []

    for i in range(n):
        min_distances_all = {
            'left_eye': [],
            'right_eye': [],
            'nose': [],
            'mouth': []
        }

        for j in range(n):
            if i == j:
                continue

            # Calcula as distâncias entre as partes faciais
            d_le = dtw_2d_curve(df_people_points.iloc[i]['left_eye_points'], df_people_points.iloc[j]['left_eye_points'])
            d_re = dtw_2d_curve(df_people_points.iloc[i]['right_eye_points'], df_people_points.iloc[j]['right_eye_points'])
            d_nose = dtw_2d_curve(df_people_points.iloc[i]['nose_points'], df_people_points.iloc[j]['nose_points'])
            d_mouth = dtw_2d_curve(df_people_points.iloc[i]['mouth_points'], df_people_points.iloc[j]['mouth_points'])

            # Armazena as distâncias junto com a tag do outro indivíduo
            tag_j = df_people_points.iloc[j]['tag']
            min_distances_all['left_eye'].append((d_le, tag_j))
            min_distances_all['right_eye'].append((d_re, tag_j))
            min_distances_all['nose'].append((d_nose, tag_j))
            min_distances_all['mouth'].append((d_mouth, tag_j))

        # Para cada parte do rosto, seleciona a menor distância e pega a tag correspondente
        selected_tags = []
        for part in ['left_eye', 'right_eye', 'nose', 'mouth']:
            min_tuple = min(min_distances_all[part], key=lambda x: x[0])
            # print(min_tuple[1].split('_')[0])
            selected_tags.append(min_tuple[1].split('_')[0])  # pega a tag associada à menor distância

        # Conta a tag mais comum entre as quatro partes
        counter = Counter(selected_tags)
        most_common_tag = counter.most_common(1)[0][0]

        # Salva o resultado para esse indivíduo
        results.append({
            'index': i,
            'most_common_tag': most_common_tag,
            'tags_used': selected_tags,
            'tag_counts': dict(counter)
        })

    return results

# Chamada da função
most_common_results = get_most_common_tag_for_each(df_people_points)


In [8]:
df_matches = pd.DataFrame(columns=['tag', 'most_common_tag', 'tags_used'])


for res in most_common_results:
    tag = df_people_points.iloc[res['index']]['tag']
    most_common_tag = res['most_common_tag']
    tags_used = res['tags_used']
    df_matches = pd.concat([df_matches, pd.DataFrame({'tag': [tag], 'most_common_tag': [most_common_tag], 'tags_used': [tags_used]})], ignore_index=True)
df_matches


,tag,most_common_tag,tags_used
0,00029_0,00029,"[00029, 00029, 00070, 00070]"
1,00029_1,00029,"[00029, 00029, 00029, 00029]"
2,00029_2,00146,"[00732, 00146, 00146, 00029]"
3,00029_3,00732,"[00732, 00029, 00732, 00732]"
4,00029_4,00146,"[00732, 00146, 00146, 00146]"
5,00029_5,00070,"[00070, 00070, 00146, 00029]"
6,00029_6,00029,"[00029, 00029, 00029, 00146]"
7,00029_7,00029,"[00029, 00070, 00029, 00029]"
8,00029_8,00732,"[00029, 00146, 00732, 00732]"
9,00029_9,00732,"[00029, 00732, 00732, 00732]"


In [10]:

matches = df_matches[df_matches['tag'].str[:5] == df_matches['most_common_tag']]

acuracia = len(matches) / len(df_matches) * 100
acuracia

37.5

In [ ]:
precision

87.5

In [10]:
len(matches)

17

In [23]:
matches

,tag,most_common_tag,tags_used
0,00029_0,00029,"[00029, 00029, 00070, 00070]"
1,00029_1,00029,"[00029, 00732, 00029, 00029]"
2,00029_2,00029,"[00029, 00732, 00029, 00029]"
3,00029_3,00029,"[00732, 00029, 00029, 00029]"
4,00029_4,00029,"[00732, 00146, 00029, 00029]"
5,00029_5,00029,"[00029, 00146, 00029, 00029]"
6,00029_6,00029,"[00029, 00029, 00029, 00029]"
7,00029_7,00029,"[00029, 00029, 00029, 00029]"
8,00029_8,00029,"[00029, 00029, 00146, 00732]"
9,00029_9,00029,"[00029, 00029, 00029, 00029]"
